## Data Prep Merge

In [8]:
import pandas as pd

# merge df_umsatz_warengruppe, df_kiel_events, df_kiwo, df_Schiffahrt, df_wetter, df_wochentage
# df_umsatz_warengruppe is the main dataframe, add the others via left merge
# all dataframes are in the folder _Variables_cleaned

df_umsatz_warengruppe = pd.read_csv('../_Variables_cleaned/df_umsatz_warengruppe.csv')
df_kiel_events = pd.read_csv('../_Variables_cleaned/df_kiel_events.csv')
df_kiwo = pd.read_csv('../_Variables_cleaned/df_kiwo.csv')
df_Schifffahrt = pd.read_csv('../_Variables_cleaned/df_Schifffahrt.csv')
df_wetter = pd.read_csv('../_Variables_cleaned/df_wetter.csv')
df_wochentage = pd.read_csv('../_Variables_cleaned/df_wochentage.csv')

df_umsatz_warengruppe = pd.merge(df_umsatz_warengruppe, df_kiel_events, on='Datum', how='left')
df_umsatz_warengruppe = pd.merge(df_umsatz_warengruppe, df_kiwo, on='Datum', how='left')
df_umsatz_warengruppe = pd.merge(df_umsatz_warengruppe, df_Schifffahrt, on='Datum', how='left')
df_umsatz_warengruppe = pd.merge(df_umsatz_warengruppe, df_wetter, on='Datum', how='left')
df_umsatz_warengruppe = pd.merge(df_umsatz_warengruppe, df_wochentage, on='Datum', how='left')

# save dataframe as df_prepared in _data_prepared
df_umsatz_warengruppe.to_csv('../_data_prepared/df_dataset_complete.csv', index=False)


Datensatz alles von 01.07.2013 löschen

In [15]:
import pandas as pd

# Load the dataframe
df = pd.read_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_dataset_complete.csv')

# Convert Datum to datetime
df['Datum'] = pd.to_datetime(df['Datum'])

# Filter dataset to keep only dates from 01.07.2013 onwards
cutoff_date = pd.to_datetime('2013-07-01')
df = df[df['Datum'] >= cutoff_date]

# Verify the date range
print("Earliest date in dataset:", df['Datum'].min())
print("Latest date in dataset:", df['Datum'].max())
print("Total rows in dataset:", len(df))

# Overwrite the original CSV file
df.to_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_dataset_complete.csv', index=False)

Earliest date in dataset: 2013-07-01 00:00:00
Latest date in dataset: 2019-08-01 00:00:00
Total rows in dataset: 250625


Imputation Median

In [17]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# Load the dataframe
df = pd.read_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_dataset_complete.csv')

# Convert Datum to datetime
df['Datum'] = pd.to_datetime(df['Datum'])

# Create mask for data up to cutoff date
cutoff_date = pd.to_datetime('2018-07-31')
mask = df['Datum'] <= cutoff_date

# Identify numerical columns
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Count missing values for numerical columns up to the cutoff date
missing_values = df.loc[mask, numerical_columns].isnull().sum()

# Calculate percentage of missing values
missing_percentage = (df.loc[mask, numerical_columns].isnull().sum() / len(df.loc[mask])) * 100

# Combine count and percentage, sort by count in descending order
missing_summary = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage (%)': missing_percentage
})

# Filter to show only columns with missing values
missing_summary = missing_summary[missing_summary['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

print("Missing Values Analysis for Numerical Columns up to 2018-07-31:")
print(missing_summary)
print("Missing Values Analysis for Data up to 2018-07-31:")
print(missing_summary)

# Set cutoff date
cutoff_date = pd.to_datetime('2018-07-31')

# Identify numerical columns automatically
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Create mask for data up to cutoff date
imputation_mask = df['Datum'] <= cutoff_date

# Print initial missing values for numerical columns
print("Missing values before imputation (up to 31.07.2018):")
print(df.loc[imputation_mask, numerical_columns].isnull().sum())

# Perform median imputation only on numerical columns up to cutoff date
median_imputer = SimpleImputer(strategy='median')
df.loc[imputation_mask, numerical_columns] = median_imputer.fit_transform(
    df.loc[imputation_mask, numerical_columns]
)

# Verify imputation results
print("\nMissing values after imputation (up to 31.07.2018):")
print(df.loc[imputation_mask, numerical_columns].isnull().sum())

print("\nMissing values after cut-off date:")
print(df.loc[~imputation_mask, numerical_columns].isnull().sum())

# Optional: Save the imputed dataframe
df.to_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_dataset_imputed.csv', index=False)

Missing Values Analysis for Numerical Columns up to 2018-07-31:
                                      Missing Count  Missing Percentage (%)
bewoelkt_bins_bewoelkt_0wenig_0_2              2411                0.963352
bewoelkt_bins_bewoelkt_1mäßig_3_5              2411                0.963352
bewoelkt_bins_bewoelkt_2stark_6_8              2411                0.963352
Bewoelkung                                     2411                0.963352
Windgeschwindigkeit                             466                0.186197
Temperatur                                      466                0.186197
Wettercode                                      466                0.186197
Monat                                           466                0.186197
Wettercode_4_gewitter                           466                0.186197
Wettercode_1_keine_beeintraechtigung            466                0.186197
Wettercode_2_leichter_niederschlag              466                0.186197
Wettercode_3_starker_nie

Feature Tag ohne Umsatz

In [33]:
import pandas as pd
import numpy as np
from workalendar.europe import Germany
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday

class GermanHolidayCalendar(AbstractHolidayCalendar):
    rules = [Holiday(name=holiday[1], year=holiday[0].year, month=holiday[0].month, day=holiday[0].day) for holiday in Germany().holidays()]

def create_holiday_feature(df):
    # Initialize holiday calendar
    cal = GermanHolidayCalendar()
    
    # Get holidays for the entire date range in the dataset
    start_year = df['Datum'].dt.year.min()
    end_year = df['Datum'].dt.year.max()
    
    holidays = cal.holidays(start=f'{start_year}-01-01', end=f'{end_year}-12-31').to_pydatetime()
    holidays = pd.DataFrame(holidays, columns=['date'])
    
    # Create holiday feature
    df['holiday_zero_sales'] = df['Datum'].isin(holidays['date']).astype(int)
    
    # Optional: Add more nuanced holiday impact
    df['holiday_type'] = np.where(
        (df['Datum'].dt.month == 12) & (df['Datum'].dt.day.isin([25, 26])), 
        'Christmas', 
        np.where(
            (df['Datum'].dt.month == 1) & (df['Datum'].dt.day == 1), 
            'NewYear',
            np.where(
                (df['Datum'].dt.month == 5) & (df['Datum'].dt.day == 1), 
                'LaborDay',
                np.where(
                    df['Datum'].isin(holidays['date']), 
                    'Other', 
                    'No Holiday'
                )
            )
        )
    )
    
    return df

# Example usage
df = pd.read_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_dataset_complete.csv')
df['Datum'] = pd.to_datetime(df['Datum'])
df = create_holiday_feature(df)
df.head()

# Save to the same file (overwrite original dataset)
df.to_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_dataset_complete.csv', index=False)

# OR save to a new file to keep the original intact
df.to_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_dataset_with_holidays.csv', index=False)

updated_df = pd.read_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_dataset_complete.csv')
print(updated_df.head())

# Drop the 'holiday_type' column
df = df.drop(columns=['holiday_type'])

# Save the updated DataFrame back to the file
df.to_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_dataset_complete.csv', index=False)

# Optional: Verify that the column has been removed
print(df.head())


        Datum      Umsatz  Warengruppe  Brot  Broetchen  Croissant  \
0  2013-07-01  148.828353          1.0   1.0        0.0        0.0   
1  2013-07-01  148.828353          1.0   1.0        0.0        0.0   
2  2013-07-01  148.828353          1.0   1.0        0.0        0.0   
3  2013-07-01  148.828353          1.0   1.0        0.0        0.0   
4  2013-07-01  148.828353          1.0   1.0        0.0        0.0   

   Konditorei  Kuchen  Saisonbrot  national_holiday  ...  \
0         0.0     0.0         0.0               0.0  ...   
1         0.0     0.0         0.0               0.0  ...   
2         0.0     0.0         0.0               0.0  ...   
3         0.0     0.0         0.0               0.0  ...   
4         0.0     0.0         0.0               0.0  ...   

   Wettercode_4_gewitter  Wochentag_Mo  Wochentag_Di  Wochentag_Mi  \
0                    0.0           1.0           0.0           0.0   
1                    0.0           1.0           0.0           0.0   
2       

## Divide dataset

In [9]:
import pandas as pd

# read data as df (df_merged_umsatz_kiwo01_wetter_wochentage.csv) in 1_DatasetCharacteristics
df = pd.read_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_dataset_complete.csv')

# Den Datensatz df teilen in Trainingsdatensatz and Validierungsdatensatz
# df_training: alle Daten bis 31.07.2017
# df_test: alle Daten vom 01.08.2017 bis 31.07.2018
df_training = df[df['Datum'] < '2017-08-01']
df_test = df[(df['Datum'] >= '2017-08-01') & (df['Datum'] <= '2018-07-31')]

# save df_training and df_test as csv files
df_training.to_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_training.csv', index=False)
df_test.to_csv('/workspaces/MA-bakery-sales-prediction/_data_prepared/df_test.csv', index=False)